<a href="https://colab.research.google.com/github/MarciaFG/skill-flow/blob/main/DEBUG_COUNTRY_Flows_1980_2022_first_level_for.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Academic Mobility Flows (1980-2022)**

# Global Talent Flows: The Academic Mobility Flows Webtool

**Author:** Marcia R. Ferreira (Complexity Science Hub Vienna & TU Wien)


*The best way to send information is to wrap it up in a person.*
    (Julius Robert Oppenheimer, 1948)


- How do academic talents travel around the world?
- Where do researchers go when they change affiliations? And where do they come from?
- Which research institutions, regions or countries are hotspots for scientific expertise in specific research areas?
- And which of these institutions are knowledge sinks? 
- Are there sinks and hotspots that researchers lose or gain disproportionate among researchers?

Welcome to the [Complexity Science Hub Vienna](https://https://www.csh.ac.at/) “Global Talent Flows” analytics notebook created in collaboration with the [Dimensions.ai](https://www.dimensions.ai/), provides insights that go beyond traditional performance indicators. Academic mobility, as measured by changes in author affiliations, is at the heart of our investigation. Thus, demonstrating that bibliometric data can provide deep insights into policy, researcher mobility, and research entities attraction strategies, all of which are still understudied aspects of Quantitative Science Studies.
Data source: The data was provided by Dimensions.ai. We welcome feedback on our data visualization, and our scientific research. Explore the high-granularity mobility of researchers (aggregated) for thousands of publication-producing institutions across the globe. 

**Disclaimer:** This data is not intended for commercialization or consulting and the data cannot be provided upon request! 

For more information about the project contact me on here: [Márcia R. Ferreira](https://www.csh.ac.at/researcher/marcia-ferreira/) 


- **Input:** Dimensions database on BigQuery
- **Output:**
Folder containing documentation about the project:
- https://drive.google.com/drive/folders/1Ac7nL2zzi8Q1crN1NaoSzzk_8YSBxD60?usp=sharing
- https://csh.ac.at/vis/skill-flow/ (deprecated)
- https://vis.csh.ac.at/new-skill-flows/ (new)
- **Github repository:** https://github.com/MarciaFG/skill-flow


## Initialization

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

/bin/bash: nvidia-smi: command not found
Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Install Drivers

In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
#!pip install pynput
#pip install plotly==5.4.0
# libraries
import psutil
import humanize
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import torch
import nltk
import GPUtil as GPU
import plotly.graph_objs as go
import plotly.io as pio

# plotting
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from google.cloud import bigquery
from google.colab import files
%load_ext google.colab.data_table
%load_ext google.cloud.bigquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=1cba6b96c1887490063e0f0d09b0cf1cf4450f5d8bcd74679ea58fc53809884e
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil


In [ ]:
# only one GPU on Colab and isn’t guaranteed
# import psutil
# import os
# import humanize
# import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ),\
       " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB"\
       .format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

**Loading data from Google Drive (If needed)**

In [ ]:
# run this to upload files
# from google.colab import files
# uploaded = files.upload() 

**Mounting the Google Drive folder**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# let's test it
#with open('/content/drive/My Drive/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/foo.txt

Mounted at /content/drive


**Runtime credentials**

In [6]:
# Provide your credentials to the runtime
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


**Declare the Cloud project ID which will be used throughout this notebook**

In [7]:
# declare your project 
project_id = "cshdimensionstest"

# START

In [ ]:
#@title  { display-mode: "both" }
#@title  { display-mode: "form" }
#@title  { vertical-output: true, display-mode: "form" }
%%html
<marquee style='width: 30%; color: blue;'><b>Whee !!</b></marquee>

# **PART I - Preprocessing**
## **1980-2022**
- Filtering by authors who have at least 2 publications 
- Filtering by authors whose first publication year is at least 1980
- Filtering by authors who have published between 1980 and 2022

## **(1) Load Data**

In [8]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.intermediary_country AS

WITH researcher_first_pub_year AS (
  SELECT
    researcher_id,
    MIN(p.year) AS first_pub_year
  FROM
    `dimensions-ai.data_analytics.publications` p,
    UNNEST(authors) AS researchers
  GROUP BY
    researcher_id
  HAVING MIN(p.year) >= 1980
),
unnested_grid_ids AS (
SELECT
  researchers.researcher_id,
  grid as grid_ids,
  p.id AS pub_id,
  p.year,
  category.id
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(authors) AS researchers,
  UNNEST(researchers.grid_ids) AS grid,
  UNNEST(category_for.first_level.full) AS category
  JOIN
    researcher_first_pub_year rp
  ON
    rp.researcher_id = researchers.researcher_id
  WHERE
    category.id IS NOT NULL                     -- only publications that have a category id
    AND researchers.researcher_id IS NOT NULL
    AND researchers.grid_ids IS NOT NULL
    AND p.year BETWEEN 1980 AND 2022            -- only publications between 1980-2022
)
SELECT DISTINCT a.*, b.country, b.country_code
FROM unnested_grid_ids a
JOIN cshdimensionstest.test.organisations_country b
ON a.grid_ids=b.id;

SELECT *
FROM cshdimensionstest.test.intermediary_country
ORDER BY researcher_id, year, pub_id, grid_ids
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_id,grid_ids,pub_id,year,id,country,country_code
0,ur.01000000010.53,grid.461843.c,pub.1077606951,2007,80003,China,CN
1,ur.01000000010.53,grid.461843.c,pub.1028748827,2011,80003,China,CN
2,ur.01000000010.53,grid.461843.c,pub.1040612737,2012,80003,China,CN
3,ur.01000000010.53,grid.461843.c,pub.1004493655,2013,80003,China,CN
4,ur.01000000010.53,grid.461843.c,pub.1039762887,2013,80003,China,CN
5,ur.01000000010.53,grid.461843.c,pub.1040126984,2013,80003,China,CN
6,ur.01000000010.53,grid.461843.c,pub.1049771699,2013,80003,China,CN
7,ur.01000000010.53,grid.461843.c,pub.1078832884,2013,80003,China,CN
8,ur.01000000010.53,grid.461843.c,pub.1121667807,2013,80003,China,CN
9,ur.01000000010.53,grid.461843.c,pub.1121815093,2013,80003,China,CN


In [ ]:
%%bigquery --project $project_id
select count(distinct researcher_id) from cshdimensionstest.test.intermediary_country -- 18394743 same as the original intermediary table

###*(1.1) Population Statistics*

In [11]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country AS

WITH total_researchers AS (
  SELECT
    country_code,
    country,
    year,
    COUNT(DISTINCT researcher_id) AS total_researchers
  FROM
    cshdimensionstest.test.intermediary_country
  GROUP BY
    country_code,
    country,
    year
),
researcher_grid_counts AS (
  SELECT
    researcher_id,
    COUNT(DISTINCT country_code) AS country_count
  FROM
    cshdimensionstest.test.intermediary
  GROUP BY
    researcher_id
),
non_mobile_researchers AS (
  SELECT
    country_code,
    country,
    year,
    COUNT(DISTINCT researcher_id) AS non_mobile_researchers
  FROM
    cshdimensionstest.test.intermediary_country
  WHERE
    researcher_id IN (SELECT researcher_id FROM researcher_grid_counts WHERE country_count = 1)
  GROUP BY
    country_code,
    country,
    year
),
mobile_researchers AS (
  SELECT
    country_code,
    country,
    year,
    COUNT(DISTINCT researcher_id) AS mobile_researchers
  FROM
    cshdimensionstest.test.intermediary_country
  WHERE
    researcher_id IN (SELECT researcher_id FROM researcher_grid_counts WHERE country_count > 1)
  GROUP BY
    country_code,
    country,
    year
)
SELECT
  tr.country_code,
  tr.country,
  tr.year,
  tr.total_researchers,
  COALESCE(nm.non_mobile_researchers, 0) AS non_mobile_researchers,
  COALESCE(mr.mobile_researchers, 0) AS mobile_researchers,
  ROUND((COALESCE(nm.non_mobile_researchers, 0) * 100.0) / tr.total_researchers, 2) AS pct_non_mobile,
  ROUND((COALESCE(mr.mobile_researchers, 0) * 100.0) / tr.total_researchers, 2) AS pct_mobile
FROM
  total_researchers tr
LEFT JOIN
  non_mobile_researchers nm
  ON tr.country_code = nm.country_code AND tr.year = nm.year
LEFT JOIN
  mobile_researchers mr
  ON tr.country_code = mr.country_code AND tr.year = mr.year
ORDER BY
  tr.country_code, tr.year;

SELECT *
FROM cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country
ORDER BY country_code, year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,country,year,total_researchers,non_mobile_researchers,mobile_researchers,pct_non_mobile,pct_mobile
0,AD,Andorra,2009,1,1,0,100.00,0.00
1,AD,Andorra,2014,2,2,0,100.00,0.00
2,AD,Andorra,2015,1,1,0,100.00,0.00
3,AD,Andorra,2017,2,1,1,50.00,50.00
4,AD,Andorra,2018,1,1,0,100.00,0.00
5,AD,Andorra,2019,1,0,1,0.00,100.00
6,AD,Andorra,2020,4,2,2,50.00,50.00
7,AD,Andorra,2021,5,4,1,80.00,20.00
8,AD,Andorra,2022,4,2,2,50.00,50.00
9,AE,United Arab Emirates,1980,4,4,0,100.00,0.00


**Selecting mobile researchers for further analysis**

In [7]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_country AS

WITH total_researchers AS (
  SELECT
    researcher_id,
    country_code,
    country,
    grid_ids,
    pub_id,
    id,
    year
  FROM
    cshdimensionstest.test.intermediary_country
),
researcher_grid_counts AS (
  SELECT
    researcher_id,
    COUNT(DISTINCT country_code) AS country_count
  FROM
    cshdimensionstest.test.intermediary
  GROUP BY
    researcher_id
)
  SELECT
    researcher_id,
    country_code,
    country,
    grid_ids,
    pub_id,
    id,
    year
  FROM
    total_researchers
  WHERE
    researcher_id IN (SELECT researcher_id FROM researcher_grid_counts WHERE country_count > 1);

SELECT *
FROM cshdimensionstest.test.au_pub_history_1980_2022_country 
ORDER BY researcher_id, year, pub_id, grid_ids, country_code
limit 1000

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_id,country_code,country,grid_ids,pub_id,id,year
0,ur.010000000201.99,TW,Taiwan,grid.260542.7,pub.1040782254,80001,2014
1,ur.010000000201.99,TW,Taiwan,grid.260542.7,pub.1040782254,80002,2014
2,ur.010000000201.99,TW,Taiwan,grid.412046.5,pub.1040782254,80002,2014
3,ur.010000000201.99,TW,Taiwan,grid.412046.5,pub.1040782254,80001,2014
4,ur.010000000201.99,TW,Taiwan,grid.453140.7,pub.1040782254,80001,2014
...,...,...,...,...,...,...,...
995,ur.01000000757.03,KR,South Korea,grid.414966.8,pub.1111918713,80003,2019
996,ur.01000000757.03,KR,South Korea,grid.411947.e,pub.1112088216,80002,2019
997,ur.01000000757.03,KR,South Korea,grid.411947.e,pub.1112088216,80003,2019
998,ur.01000000757.03,KR,South Korea,grid.411947.e,pub.1112506102,80003,2019


In [ ]:
%%bigquery --project $project_id
SELECT COUNT(DISTINCT country_code) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; -- 2,037,951


- SELECT COUNT(DISTINCT researcher_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; -- 2,037,951
- SELECT COUNT(DISTINCT grid_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; --56,221
- SELECT COUNT(DISTINCT pub_id) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; --32,852,373
- SELECT COUNT(DISTINCT country_Code) FROM cshdimensionstest.test.au_pub_history_1980_2022_country; --217


## (2) Trajectory Sequence

1.   We opt for a simplified career trajectory
2.   We take into account the first and last year of publication at a given country







###*(2.1) Simplified*

In [20]:
%%bigquery --project $project_id 

CREATE OR REPLACE TABLE cshdimensionstest.test.simple_sequence_1980_2022_country AS

SELECT DISTINCT 
    researcher_id as researcher_ids,
    country_code,
    country,
    COUNT(DISTINCT pub_id) AS t_pubs,
    MIN(year) AS start_year,
    MAX(year) AS end_year,
    IFNULL(dense_rank() OVER (
        PARTITION BY researcher_id 
        ORDER BY MIN(year), MAX(year) ASC
    ), 0) AS t
FROM `cshdimensionstest.test.au_pub_history_1980_2022_country`
GROUP BY researcher_id, country_code, country;

# Let's check the table
# ur.013012771111.87
# Rodrigo Costas

SELECT * 
FROM cshdimensionstest.test.simple_sequence_1980_2022_country
WHERE researcher_ids = 'ur.013012771111.87' 
ORDER BY t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,country_code,country,t_pubs,start_year,end_year,t
0,ur.013012771111.87,ES,Spain,3,2007,2010,1
1,ur.013012771111.87,NL,Netherlands,82,2009,2022,2
2,ur.013012771111.87,ZA,South Africa,35,2017,2022,3


**ok we are good to go!**

# **PART II - Mobility Network**

## (3) Mobility Flows

We will split the calculation of the network flows:
1. Institutions
2. Cities
3. Countries
4. FOR 2-digit field
- This script is at the institutional level
---



We can later on think of costumisable layers such as NUTS2 etc

### ***(3.1) Author Country Flows***
In this part we focus on flows at the level of institutions

  **ATT: This is the most computationally expensive table, becareful with running it too many times**

In [21]:
%%bigquery --project $project_id 

CREATE TABLE cshdimensionstest.test.simple_sequence_1980_2022_clustered_country 
(
  researcher_ids STRING,
  country_code STRING,
  start_year INT64,
  t INT64
)
CLUSTER BY researcher_ids
OPTIONS(
  description="A clustered table of simple_sequence_1980_2022"
)
AS
SELECT researcher_ids, country_code, start_year, t 
FROM cshdimensionstest.test.simple_sequence_1980_2022_country;

# Now we have everything we need to construct the flows at the institutional level
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country AS
SELECT 
    a.researcher_ids,
    a.country_code AS unit1,
    b.country_code AS unit2,
    a.start_year AS p1,
    b.start_year AS p2
FROM
    cshdimensionstest.test.simple_sequence_1980_2022_clustered_country a 
    INNER JOIN cshdimensionstest.test.simple_sequence_1980_2022_clustered_country b 
    ON a.researcher_ids = b.researcher_ids 
WHERE a.t = b.t - 1;

# Check the table
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_country
WHERE researcher_ids = 'ur.013012771111.87' 
ORDER BY researcher_ids, p1, p2, unit1, unit2 
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2
0,ur.013012771111.87,ES,NL,2007,2009
1,ur.013012771111.87,NL,ZA,2009,2017


In [ ]:
# Check the table
# how many edges does it have?
%%bigquery --project $project_id 
SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_country; -- 3,031,288

 ### ***(3.2) Cross-Country Flows***
 
 Are the flows exchanged between two countries at a given calendar year

In [ ]:
%%bigquery --project $project_id

-- Calculate the total flows between institutional pairs
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_flows AS
SELECT 
  unit1 AS geoid_o,
  unit2 AS geoid_d,
  p2 AS date_d,
  COUNT(DISTINCT researcher_ids) AS total_flows, # author flows
FROM cshdimensionstest.test.flows_1980_2022_country
GROUP BY 
  geoid_o, 
  geoid_d, 
  date_d;

-- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_country_flows 
ORDER BY 
  geoid_o, 
  geoid_d, 
  date_d 
LIMIT 50;


### ***(3.3) Total Flows by Country***

***Overall flows by country over time***
- outgoing flows (counts and percentage)
- incoming flows (counts and percentage)
- total flows (counts)
- total net flows (counts)
- net_mobility_rate


In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_total_flows_agg AS
SELECT
  node,
  date_d,
  SUM(outgoing_flows) AS outgoing_flows,
  ROUND(IFNULL(SUM(outgoing_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 0), 1) AS percentage_outflows,
  SUM(incoming_flows) AS incoming_flows,
  ROUND(IFNULL(SUM(incoming_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 0), 1) AS percentage_inflows,
  SUM(outgoing_flows) + SUM(incoming_flows) AS total_flows,
  SUM(incoming_flows) - SUM(outgoing_flows) AS total_net_flows,
  ROUND(IFNULL((SUM(incoming_flows) - SUM(outgoing_flows)) / (SUM(incoming_flows) + SUM(outgoing_flows)) * 100, 0), 1) AS net_mobility_rate
FROM (
  SELECT
    unit1 AS node,
    p2 as date_d,
    COUNT(DISTINCT researcher_ids) AS outgoing_flows,
    0 AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022_country
  GROUP BY
    unit1, p2

  UNION ALL

  SELECT
    unit2 AS node,
    p2 as date_d,
    0 AS outgoing_flows,
    COUNT(DISTINCT researcher_ids) AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022_country
  GROUP BY
    unit2, p2
) 
AS flows
GROUP BY
  node, date_d;

-- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_country_total_flows_agg
ORDER BY 
  node, 
  date_d 
LIMIT 50;

In [25]:
%%bigquery --project $project_id
-- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_country_total_flows_agg
ORDER BY 
  node, 
  date_d 
LIMIT 1000;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate
0,AD,2017,0,0.0,1,100.0,1,1,100.0
1,AD,2020,0,0.0,1,100.0,1,1,100.0
2,AD,2022,0,0.0,1,100.0,1,1,100.0
3,AE,1986,0,0.0,2,100.0,2,2,100.0
4,AE,1987,0,0.0,1,100.0,1,1,100.0
...,...,...,...,...,...,...,...,...,...
995,CA,1982,116,47.2,130,52.8,246,14,5.7
996,CA,1983,184,48.5,195,51.5,379,11,2.9
997,CA,1984,232,43.3,304,56.7,536,72,13.4
998,CA,1985,319,45.2,386,54.8,705,67,9.5


## (4) Indicators

### ***(4.1) Academic Age***
The average academic age of inflowing and outflowing researchers for each node and year by joining the two subsets of data on the researcher ID

In [26]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_country AS

WITH academic_age AS (
  SELECT 
    a.researcher_ids,
    b.unit1,
    b.unit2,
    a.min_academic_age,
    b.last_year_at_unit1,
    (b.last_year_at_unit1 - a.min_academic_age + 1) AS outgoing_academic_age,
    (b.last_year_at_unit2 - a.min_academic_age + 1) AS incoming_academic_age
  FROM (
    SELECT researcher_ids, MIN(p1) AS min_academic_age
    FROM cshdimensionstest.test.flows_1980_2022_country
    GROUP BY researcher_ids
  ) AS a
  JOIN (
    SELECT 
      researcher_ids, 
      unit1, 
      unit2,
      MAX(p2) AS last_year_at_unit1,
      MAX(p2) AS last_year_at_unit2
    FROM cshdimensionstest.test.flows_1980_2022_country
    GROUP BY researcher_ids, unit1, unit2
  ) AS b
  ON a.researcher_ids = b.researcher_ids
)
SELECT * 
FROM academic_age;

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_1_country AS

SELECT a.*
  , IFNULL(b.outgoing_academic_age, 0) AS outgoing_academic_age
  , IFNULL(b.incoming_academic_age, 0) AS incoming_academic_age
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_age_country b 
  ON a.researcher_ids=b.researcher_ids 
  AND a.unit1=b.unit1 
  AND a.unit2=b.unit2;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_age_1_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY researcher_ids, p1, p1, unit1, unit2
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,outgoing_academic_age,incoming_academic_age
0,ur.013012771111.87,ES,NL,2007,2009,3,3
1,ur.013012771111.87,NL,ZA,2009,2017,11,11


In [ ]:
"""
%%bigquery --project $project_id
SELECT AVG(incoming_academic_age) from cshdimensionstest.test.flows_1980_2022_age_1; -- mean = 9.6
SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_age_1; -- rows = 17,385,293
"""

#### *(4.1.1) Academic Age Groups*

In [30]:
%%bigquery --project $project_id

-- Create non-overlapping deciles of age
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_deciles_country AS

WITH researcher_deciles AS 
(
  SELECT DISTINCT researcher_ids, incoming_academic_age,
         NTILE(10) OVER(ORDER BY incoming_academic_age) as incoming_age_decile,
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country
  GROUP BY researcher_ids, incoming_academic_age
), decile_range AS (
    SELECT 
      incoming_age_decile
    , MIN(incoming_academic_age) AS min
    , MAX(incoming_academic_age) AS max
  FROM researcher_deciles
  GROUP BY incoming_age_decile
)
, adjusted_decile_range AS (
    SELECT
      incoming_age_decile
    , CASE WHEN incoming_age_decile = 1 THEN min
           ELSE LAG(max) OVER (ORDER BY incoming_age_decile) + 1 END AS adjusted_min
    , max AS adjusted_max
    FROM decile_range
)
, deciles_joined AS (
  SELECT DISTINCT a.*, CONCAT('Age group id: ', b.incoming_age_decile,' | Range (years): ',  adjusted_min, '-', adjusted_max) age_group
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country a
  JOIN researcher_deciles b 
    ON a.researcher_ids=b.researcher_ids 
    AND a.incoming_academic_age=b.incoming_academic_age
  JOIN adjusted_decile_range c 
    ON b.incoming_age_decile=c.incoming_age_decile
)
SELECT * FROM deciles_joined;

-- Check table:
SELECT * FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
WHERE researcher_ids = 'ur.013012771111.87'
ORDER BY researcher_ids, p1, p2, unit1, unit1
LIMIT 20;


Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,outgoing_academic_age,incoming_academic_age,age_group
0,ur.013012771111.87,ES,NL,2007,2009,3,3,Age group id: 3 | Range (years): 4-4
1,ur.013012771111.87,NL,ZA,2009,2017,11,11,Age group id: 8 | Range (years): 10-12


In [28]:
%%bigquery --project $project_id
SELECT COUNT(distinct researcher_ids) as n_researchers, age_group 
FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
group by age_group
Order by age_group;

Query is running:   0%|          |

Downloading:   0%|          |

,n_researchers,age_group
0,255292,Age group id: 1 | Range (years): 1-2
1,181227,Age group id: 10 | Range (years): 18-43
2,255091,Age group id: 2 | Range (years): 3-3
3,254985,Age group id: 3 | Range (years): 4-4
4,254856,Age group id: 4 | Range (years): 5-5
5,254212,Age group id: 5 | Range (years): 6-6
6,253378,Age group id: 6 | Range (years): 7-7
7,250792,Age group id: 7 | Range (years): 8-9
8,244909,Age group id: 8 | Range (years): 10-12
9,228660,Age group id: 9 | Range (years): 13-17


**We have the following age groups**


|index|age\_group|
|---|---|
|0|Age group id: 1 &#124; Range \(years\): 1-2|
|1|Age group id: 10 &#124; Range \(years\): 18-43|
|2|Age group id: 2 &#124; Range \(years\): 3-3|
|3|Age group id: 3 &#124; Range \(years\): 4-4|
|4|Age group id: 4 &#124; Range \(years\): 5-5|
|5|Age group id: 5 &#124; Range \(years\): 6-6|
|6|Age group id: 6 &#124; Range \(years\): 7-7|
|7|Age group id: 7 &#124; Range \(years\): 8-9|
|8|Age group id: 8 &#124; Range \(years\): 10-12|
|9|Age group id: 9 &#124; Range \(years\): 13-17|


In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_groups_country AS

WITH outgoing AS (
SELECT unit1 as node, p2 as date_d, 
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 1 | Range (years): 1-2') AS outgoing_age_1_2_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 2 | Range (years): 3-3') AS outgoing_age_3_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 3 | Range (years): 4-4') AS outgoing_age_4_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 4 | Range (years): 5-5') AS outgoing_age_5_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 5 | Range (years): 6-6') AS outgoing_age_6_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 6 | Range (years): 7-7') AS outgoing_age_7_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 7 | Range (years): 8-9') AS outgoing_age_8_9_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 8 | Range (years): 10-12') AS outgoing_age_10_12_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 9 | Range (years): 13-17') AS outgoing_age_13_17_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 10 | Range (years): 18-43') AS outgoing_age_18_43_y
     FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
GROUP BY unit1, p2 )
, incoming AS (
  SELECT unit2 as node, p2 as date_d, 
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 1 | Range (years): 1-2') AS incoming_age_1_2_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 2 | Range (years): 3-3') AS incoming_age_3_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 3 | Range (years): 4-4') AS incoming_age_4_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 4 | Range (years): 5-5') AS incoming_age_5_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 5 | Range (years): 6-6') AS incoming_age_6_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 6 | Range (years): 7-7') AS incoming_age_7_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 7 | Range (years): 8-9') AS incoming_age_8_9_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 8 | Range (years): 10-12') AS incoming_age_10_12_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 9 | Range (years): 13-17') AS incoming_age_13_17_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 10 | Range (years): 18-43') AS incoming_age_18_43_y
     FROM cshdimensionstest.test.flows_1980_2022_age_deciles_country
GROUP BY unit2, date_d
) ,
age_aggregated AS 
(
  SELECT a.*, b.incoming_age_1_2_y, b.incoming_age_3_y, b.incoming_age_4_y,
  b.incoming_age_5_y, b.incoming_age_6_y, b.incoming_age_7_y, b.incoming_age_8_9_y,
  b.incoming_age_10_12_y, b.incoming_age_13_17_y, b.incoming_age_18_43_y
  FROM outgoing a
  LEFT JOIN incoming b 
    ON a.node=b.node 
    AND a.date_d=b.date_d
)
SELECT *
FROM age_aggregated
ORDER BY node, date_d;

-- Check table:
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_groups_country
ORDER BY node, date_d
LIMIT 50;

#### *(4.1.2) Academic Age Statistics*

In [32]:
# Let's average the academic age at the level of institutions
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_statistics_country AS

WITH outgoing AS (
  SELECT 
    unit1 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(outgoing_academic_age), 0) AS sum_outgoing_academic_age,
    ROUND(IFNULL(STDDEV(outgoing_academic_age), 0), 3) AS stdev_outgoing_academic_age,
    ROUND(IFNULL(APPROX_QUANTILES(outgoing_academic_age, 3)[OFFSET(1)], 0), 3) AS median_outgoing_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country
  GROUP BY unit1, p2
),
incoming AS (
  SELECT 
    unit2 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(incoming_academic_age), 0) AS sum_incoming_academic_age,
    ROUND(IFNULL(STDDEV(incoming_academic_age), 0), 3) AS stdev_incoming_academic_age,
    ROUND(IFNULL(APPROX_QUANTILES(incoming_academic_age, 3)[OFFSET(1)], 0), 3) AS median_incoming_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_1_country
  GROUP BY unit2, p2
),
age_aggregated AS (
  SELECT 
    a.node,
    a.move_year,
    IFNULL(a.num_researchers, 0) as outflows,
    IFNULL(b.num_researchers, 0) as inflows,
    ROUND(IFNULL(a.sum_outgoing_academic_age / a.num_researchers, 0), 3) AS mean_outgoing_academic_age,
    ROUND(IFNULL(a.sum_outgoing_academic_age / (a.num_researchers * a.num_researchers), 0), 3) AS mean_normalized_outgoing_academic_age,
    IFNULL(median_outgoing_academic_age, 0) AS median_outgoing_academic_age,
    IFNULL(stdev_outgoing_academic_age, 0) AS stdev_outgoing_academic_age,
    ROUND(IFNULL(b.sum_incoming_academic_age / b.num_researchers, 0), 3) AS mean_incoming_academic_age,
    ROUND(IFNULL(b.sum_incoming_academic_age / (b.num_researchers * b.num_researchers), 0), 3) AS mean_normalized_incoming_academic_age,
    IFNULL(median_incoming_academic_age, 0) AS median_incoming_academic_age,
    IFNULL(stdev_incoming_academic_age, 0) AS stdev_incoming_academic_age
  FROM outgoing a
  LEFT JOIN incoming b
  ON a.node = b.node AND a.move_year = b.move_year
)
SELECT * 
FROM age_aggregated
ORDER BY node, move_year;


SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_statistics_country
ORDER BY node, move_year
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,outflows,inflows,mean_outgoing_academic_age,mean_normalized_outgoing_academic_age,median_outgoing_academic_age,stdev_outgoing_academic_age,mean_incoming_academic_age,mean_normalized_incoming_academic_age,median_incoming_academic_age,stdev_incoming_academic_age
0,AE,1988,1,3,5.000,5.000,5.0,0.000,4.000,1.333,2.0,1.732
1,AE,1990,1,2,1.000,1.000,1.0,0.000,3.500,1.750,1.0,3.536
2,AE,1991,4,11,3.750,0.938,2.0,2.630,6.000,0.545,4.0,3.435
3,AE,1992,6,28,4.333,0.722,2.0,4.457,7.393,0.264,5.0,3.531
4,AE,1993,4,15,2.250,0.563,2.0,1.258,6.600,0.440,4.0,3.355
...,...,...,...,...,...,...,...,...,...,...,...,...
95,AM,1991,11,14,5.727,0.521,5.0,2.573,3.786,0.270,2.0,3.191
96,AM,1992,7,7,6.000,0.857,4.0,3.916,3.714,0.531,3.0,1.799
97,AM,1993,12,4,6.750,0.563,3.0,4.003,5.500,1.375,5.0,2.082
98,AM,1994,23,10,5.652,0.246,3.0,3.737,9.800,0.980,6.0,2.872


<p>The outgoing_academic_age represents the average academic age of researchers who are leaving the node (i.e., the institution or country) during the specified year. It is calculated by taking the average of the difference between the year of departure (p2) and the minimum academic age of the researcher (min_academic_age) for each unique combination of unit1 (i.e., the node) and p2 (i.e., the year). A lower value for outgoing_academic_age would indicate that researchers leaving the node are generally younger and/or have spent less time in academia, while a higher value would indicate the opposite.</p>
<p>This code also computes the average normalized academic ages of outgoing and incoming researchers by dividing the academic age by the total number of flows. The outgoing_flows and incoming_flows columns are added to the academic_age CTE to capture the total number of flows for each researcher in each year. The mean_normalized_outgoing_academic_age and mean_normalized_incoming_academic_age columns then compute the average normalized academic age by taking the average of the normalized academic age values.</p>

A positive value would indicate an increase in the average academic age, while a negative value would indicate a decrease. This measure would be less sensitive to changes in the number of flows than the absolute change in the average academic age.

<div>
  <h3>How to Interpret the Mean Normalized Outgoing Academic Age:</h3>
  <ul>
    <li>The "Normalized Outgoing Academic Age" is the average age of outgoing researchers, adjusted by the total number of outgoing flows.</li>
    <li>A higher value for the "Normalized Outgoing Academic Age" indicates that, on average, researchers leaving the institution are older, relative to the total number of outgoing flows.</li>
    <li>A lower value for the "Normalized Outgoing Academic Age" indicates that, on average, researchers leaving the institution are younger, relative to the total number of outgoing flows.</li>
  </ul>
</div>


In [36]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_academic_age_statistics_lagged AS

WITH academic_age AS 
(
  SELECT 
    node,
    move_year,
    mean_incoming_academic_age,
    mean_outgoing_academic_age,
    LAG(mean_incoming_academic_age) OVER (PARTITION BY node ORDER BY move_year) AS prev_mean_incoming_academic_age,
    LAG(mean_outgoing_academic_age) OVER (PARTITION BY node ORDER BY move_year) AS prev_mean_outgoing_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_statistics_country
  )
SELECT
  node,
  move_year,
  ROUND(IFNULL(LEAST(((mean_incoming_academic_age - prev_mean_incoming_academic_age) / NULLIF(prev_mean_incoming_academic_age, 0)) * 100, 100), 0), 3) AS incoming_academic_age_pct_change,
  ROUND(IFNULL(LEAST(((mean_outgoing_academic_age - prev_mean_outgoing_academic_age) / NULLIF(prev_mean_outgoing_academic_age, 0)) * 100, 100), 0), 3) AS outgoing_academic_age_pct_change
FROM academic_age;


CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_academic_age_stats AS

SELECT a.*
  , IFNULL(b.incoming_academic_age_pct_change, 0) AS incoming_academic_age_pct_change
  , IFNULL(b.outgoing_academic_age_pct_change, 0) AS outgoing_academic_age_pct_change
FROM cshdimensionstest.test.flows_1980_2022_age_statistics_country a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_country_academic_age_statistics_lagged b
  ON a.node=b.node 
  AND a.move_year=b.move_year;

-- Check table:
SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_academic_age_stats
ORDER BY node, move_year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,outflows,inflows,mean_outgoing_academic_age,mean_normalized_outgoing_academic_age,median_outgoing_academic_age,stdev_outgoing_academic_age,mean_incoming_academic_age,mean_normalized_incoming_academic_age,median_incoming_academic_age,stdev_incoming_academic_age,incoming_academic_age_pct_change,outgoing_academic_age_pct_change
0,AE,1988,1,3,5.000,5.000,5.0,0.000,4.000,1.333,2.0,1.732,0.000,0.000
1,AE,1990,1,2,1.000,1.000,1.0,0.000,3.500,1.750,1.0,3.536,-12.500,-80.000
2,AE,1991,4,11,3.750,0.938,2.0,2.630,6.000,0.545,4.0,3.435,71.429,100.000
3,AE,1992,6,28,4.333,0.722,2.0,4.457,7.393,0.264,5.0,3.531,23.217,15.547
4,AE,1993,4,15,2.250,0.563,2.0,1.258,6.600,0.440,4.0,3.355,-10.726,-48.073
5,AE,1994,8,25,6.750,0.844,6.0,3.059,7.360,0.294,6.0,3.569,11.515,100.000
6,AE,1995,10,15,7.500,0.750,4.0,5.297,8.733,0.582,7.0,4.543,18.655,11.111
7,AE,1996,13,21,5.538,0.426,3.0,4.034,8.381,0.399,7.0,4.364,-4.031,-26.160
8,AE,1997,49,88,2.347,0.048,1.0,3.106,7.034,0.080,4.0,3.859,-16.072,-57.620
9,AE,1998,19,23,6.684,0.352,4.0,4.042,9.217,0.401,5.0,4.945,31.035,100.000


The formula ((new_value / old_value) - 1) * 100 is used to calculate percentage change between two values. The result will be a positive or negative number representing the percentage change between the new value and the old value.

If the result is positive, it indicates that the new value is higher than the old value, and the percentage change represents the increase. For example, if the old value is 100 and the new value is 120, the percentage change would be ((120/100) - 1) * 100 = 20, indicating a 20% increase.

If the result is negative, it indicates that the new value is lower than the old value, and the percentage change represents the decrease. For example, if the old value is 120 and the new value is 100, the percentage change would be ((100/120) - 1) * 100 = -16.67, indicating a 16.67% decrease.


In [37]:
# lets merge with the age stats
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_flow_statistics AS

SELECT 
  a.*, 
  IFNULL(mean_outgoing_academic_age, 0) AS mean_outgoing_academic_age,
  IFNULL(stdev_outgoing_academic_age, 0) AS stdev_outgoing_academic_age,
  IFNULL(mean_normalized_outgoing_academic_age, 0) AS mean_normalized_outgoing_academic_age,
  IFNULL(median_outgoing_academic_age, 0) AS median_outgoing_academic_age,
  IFNULL(outgoing_academic_age_pct_change, 0) AS outgoing_academic_age_pct_change,
  IFNULL(mean_incoming_academic_age, 0) AS mean_incoming_academic_age,
  IFNULL(stdev_incoming_academic_age, 0) AS stdev_incoming_academic_age,
  IFNULL(mean_normalized_incoming_academic_age, 0) AS mean_normalized_incoming_academic_age,
  IFNULL(median_incoming_academic_age, 0) AS median_incoming_academic_age,
  IFNULL(incoming_academic_age_pct_change, 0) AS incoming_academic_age_pct_change,
FROM 
  cshdimensionstest.test.flows_1980_2022_country_total_flows_agg a
LEFT JOIN 
  cshdimensionstest.test.flows_1980_2022_country_academic_age_stats b
ON 
  a.node=b.node 
  AND a.date_d=b.move_year;

-- Check table:
SELECT * 
FROM 
  cshdimensionstest.test.flows_1980_2022_country_flow_statistics
ORDER BY 
  node, 
  date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate,mean_outgoing_academic_age,stdev_outgoing_academic_age,mean_normalized_outgoing_academic_age,median_outgoing_academic_age,outgoing_academic_age_pct_change,mean_incoming_academic_age,stdev_incoming_academic_age,mean_normalized_incoming_academic_age,median_incoming_academic_age,incoming_academic_age_pct_change
0,AD,2017,0,0.0,1,100.0,1,1,100.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000
1,AD,2020,0,0.0,1,100.0,1,1,100.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000
2,AD,2022,0,0.0,1,100.0,1,1,100.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000
3,AE,1986,0,0.0,2,100.0,2,2,100.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000
4,AE,1987,0,0.0,1,100.0,1,1,100.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000
5,AE,1988,1,25.0,3,75.0,4,2,50.0,5.000,0.000,5.000,5.0,0.000,4.000,1.732,1.333,2.0,0.000
6,AE,1989,0,0.0,4,100.0,4,4,100.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000
7,AE,1990,1,33.3,2,66.7,3,1,33.3,1.000,0.000,1.000,1.0,-80.000,3.500,3.536,1.750,1.0,-12.500
8,AE,1991,4,26.7,11,73.3,15,7,46.7,3.750,2.630,0.938,2.0,100.000,6.000,3.435,0.545,4.0,71.429
9,AE,1992,6,17.6,28,82.4,34,22,64.7,4.333,4.457,0.722,2.0,15.547,7.393,3.531,0.264,5.0,23.217


In [25]:
import pandas as pd
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import io

# Authenticate and create the PyDrive client.
auth.authenticate_user()

# Build the Google Drive API client
drive_service = build("drive", "v3", credentials=GoogleCredentials.get_application_default())

file_id = '12bA1nGGzfIyGNZv99HYM9GTbYIU346UX8fCQ3r1eMTo'

try:
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/plain')
    downloaded = io.StringIO(request.execute().decode('utf-8'))
    print("File downloaded successfully")
except HttpError as error:
    print(f"An error occurred: {error}")
    downloaded = None

# Read the file into a pandas DataFrame
countries = pd.read_csv(downloaded, sep='\s+', engine='python', error_bad_lines=False)
print(countries.head(10))
print(countries.dtypes)

File downloaded successfully
  country   latitude   longitude         name
0      AD  42.546245    1.601554      Andorra
1      AF  33.939110   67.709953  Afghanistan
2      AI  18.220554  -63.068615     Anguilla
3      AL  41.153332   20.168331      Albania
4      AM  40.069099   45.038189      Armenia
5      AO -11.202692   17.873887       Angola
6      AQ -75.250973   -0.071389   Antarctica
7      AR -38.416097  -63.616672    Argentina
8      AT  47.516231   14.550072      Austria
9      AU -25.274398  133.775136    Australia
country       object
latitude     float64
longitude    float64
name          object
dtype: object


<ipython-input-25-0f244c52901d>:24: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  countries = pd.read_csv(downloaded, sep='\s+', engine='python', error_bad_lines=False)
Skipping line 3: Expected 4 fields in line 3, saw 6. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 5: Expected 4 fields in line 5, saw 6. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 9: Expected 4 fields in line 9, saw 5. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 13: Expected 4 fields in line 13, saw 5. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 18: Expected 4 fields in line 18, saw 6. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 22: Expe

In [26]:
countries.head(10)

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AF,33.939110,67.709953,Afghanistan
2,AI,18.220554,-63.068615,Anguilla
3,AL,41.153332,20.168331,Albania
4,AM,40.069099,45.038189,Armenia
5,AO,-11.202692,17.873887,Angola
6,AQ,-75.250973,-0.071389,Antarctica
7,AR,-38.416097,-63.616672,Argentina
8,AT,47.516231,14.550072,Austria
9,AU,-25.274398,133.775136,Australia


In [29]:
# storing the country coordinates dataframe into GBQ for further use
!pip install pandas-gbq
import pandas_gbq

# Set your Google Cloud Platform project ID, dataset ID, and table ID
project_id = "cshdimensionstest"
dataset_id = "test"
table_id = "country_coordinates"

# Fully-qualified table name
destination_table = f"{dataset_id}.{table_id}"

# Store the DataFrame into GBQ
countries.to_gbq(destination_table, project_id, if_exists='replace')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


100%|██████████| 1/1 [00:00<00:00, 5966.29it/s]


In [28]:
%%bigquery --project $project_id 
select * from cshdimensionstest.test.country_coordinates

Query is running:   0%|          |

Downloading:   0%|          |

,country,latitude,longitude,name
0,GN,9.945587,-9.696645,Guinea
1,KI,-3.370417,-168.734039,Kiribati
2,LR,6.428055,-9.429499,Liberia
3,ME,42.708678,19.374390,Montenegro
4,PW,7.514980,134.582520,Palau
...,...,...,...,...
172,SD,12.862807,30.217636,Sudan
173,NE,17.607789,8.081666,Niger
174,SC,-4.679574,55.491977,Seychelles
175,TO,-21.178986,-175.198242,Tonga


#### *(4.1.3) Age and Distance Statistics*

In [30]:
%%bigquery --project $project_id 

# SOURCE DISTANCES
CREATE OR REPLACE TABLE cshdimensionstest.test.average_distance_source_country AS

WITH distance AS (
  SELECT 
    a.researcher_ids,
    a.unit1 as source_country,
    a.unit2 as destination_country,
    a.p2 as move_year,
    b.latitude as source_latitude,
    b.longitude as source_longitude,
    c.latitude as destination_latitude,
    c.longitude as destination_longitude,
    6371 * 2 * ASIN(SQRT(POWER(SIN(((c.latitude - b.latitude) * ACOS(-1) / 180) / 2), 2) +
      COS((b.latitude * ACOS(-1) / 180)) * COS((c.latitude * ACOS(-1) / 180)) *
      POWER(SIN(((c.longitude - b.longitude) * ACOS(-1) / 180) / 2), 2))) AS distance_km
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.country_coordinates b on a.unit1=b.country
LEFT JOIN cshdimensionstest.test.country_coordinates c on a.unit2=c.country
)
, distance_summary AS (
SELECT 
  source_country
, CAST(move_year AS INT) move_year
, AVG(distance_km) as average_distance_travelled_km
, COUNT(distinct researcher_ids) as outflows
FROM distance
GROUP BY source_country, move_year )
SELECT 
  source_country,
  move_year,
  average_distance_travelled_km,
  outflows,
  CASE
      WHEN average_distance_travelled_km <  100 Then '< 100 km'
      WHEN average_distance_travelled_km >= 100 AND average_distance_travelled_km < 1000 Then '≥ 100 km < 1000 km'
      WHEN average_distance_travelled_km >= 1000 AND average_distance_travelled_km < 5000 Then '≥ 1000 km < 5000 km'
      WHEN average_distance_travelled_km >= 5000 AND average_distance_travelled_km < 10000 Then '≥ 5000 km < 10000 km'
      WHEN average_distance_travelled_km >= 10000 AND average_distance_travelled_km < 20000 Then '≥ 10000 km < 20000 km'
      ELSE '≥ 20000 km'
  END AS distance_category
FROM distance_summary
ORDER BY source_country, move_year;

# DESTINATION DISTANCES
CREATE OR REPLACE TABLE cshdimensionstest.test.average_distance_destination_country AS

WITH distance AS (
  SELECT 
    a.researcher_ids,
    a.unit1 as source_country,
    a.unit2 as destination_country,
    a.p2 as move_year,
    b.latitude as source_latitude,
    b.longitude as source_longitude,
    c.latitude as destination_latitude,
    c.longitude as destination_longitude,
    6371 * 2 * ASIN(SQRT(POWER(SIN(((c.latitude - b.latitude) * ACOS(-1) / 180) / 2), 2) +
      COS((b.latitude * ACOS(-1) / 180)) * COS((c.latitude * ACOS(-1) / 180)) *
      POWER(SIN(((c.longitude - b.longitude) * ACOS(-1) / 180) / 2), 2))) AS distance_km
FROM cshdimensionstest.test.flows_1980_2022_country a
LEFT JOIN cshdimensionstest.test.country_coordinates b on a.unit1=b.country
LEFT JOIN cshdimensionstest.test.country_coordinates c on a.unit2=c.country
), distance_summary AS (
SELECT 
  destination_country
, CAST(move_year AS INT) move_year
, AVG(distance_km) as average_distance_travelled_km
, COUNT(distinct researcher_ids) as inflows
FROM distance
GROUP BY destination_country, move_year )
SELECT 
  destination_country,
  move_year,
  average_distance_travelled_km,
  inflows,
  CASE
      WHEN average_distance_travelled_km <  100 Then '< 100 km'
      WHEN average_distance_travelled_km >= 100 AND average_distance_travelled_km < 1000 Then '≥ 100 km < 1000 km'
      WHEN average_distance_travelled_km >= 1000 AND average_distance_travelled_km < 5000 Then '≥ 1000 km < 5000 km'
      WHEN average_distance_travelled_km >= 5000 AND average_distance_travelled_km < 10000 Then '≥ 5000 km < 10000 km'
      WHEN average_distance_travelled_km >= 10000 AND average_distance_travelled_km < 20000 Then '≥ 10000 km < 20000 km'
      ELSE '≥ 20000 km'
  END AS distance_category
FROM distance_summary
ORDER BY destination_country, move_year;


# Merging distances and age and flow statistics for instutitions
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_distances AS

SELECT 
  a.*,
  IFNULL(b.average_distance_travelled_km, 0) AS mean_distance_travelled_to_destination,
  IFNULL(b.distance_category, '') AS distance_category_to_destination,
  IFNULL(c.average_distance_travelled_km, 0) AS mean_distance_travelled_from_source,
  IFNULL(c.distance_category, '') AS distance_category_from_source,
  IFNULL(d.total_researchers, 0) AS total_researchers,
  IFNULL(d.non_mobile_researchers, 0) AS non_mobile_researchers,
  IFNULL(d.pct_non_mobile, 0) AS pct_non_mobile,
  IFNULL(d.mobile_researchers, 0) AS mobile_researchers,
  IFNULL(d.pct_mobile, 0) AS pct_mobile
FROM cshdimensionstest.test.flows_1980_2022_country_flow_statistics a
LEFT JOIN cshdimensionstest.test.average_distance_destination_country b 
  ON a.node = b.destination_country AND a.date_d = b.move_year
LEFT JOIN cshdimensionstest.test.average_distance_source_country c 
  ON a.node = c.source_country AND a.date_d = c.move_year
LEFT JOIN cshdimensionstest.test.au_pub_history_1980_2022_population_statistics_country d
 ON a.node = d.country_code and a.date_d =d.year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_distances
ORDER BY node, date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate,mean_outgoing_academic_age,...,incoming_academic_age_pct_change,mean_distance_travelled_to_destination,distance_category_to_destination,mean_distance_travelled_from_source,distance_category_from_source,total_researchers,non_mobile_researchers,pct_non_mobile,mobile_researchers,pct_mobile
0,AD,2017,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,2,1,50.00,1,50.00
1,AD,2020,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,502.025372,≥ 100 km < 1000 km,0.000000,,4,2,50.00,2,50.00
2,AD,2022,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,502.025372,≥ 100 km < 1000 km,0.000000,,4,2,50.00,2,50.00
3,AE,1986,0,0.0,2,100.0,2,2,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,8,2,25.00,6,75.00
4,AE,1987,0,0.0,1,100.0,1,1,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,10,5,50.00,5,50.00
5,AE,1988,1,25.0,3,75.0,4,2,50.0,5.000,...,0.000,0.000000,≥ 20000 km,0.000000,≥ 20000 km,18,11,61.11,7,38.89
6,AE,1989,0,0.0,4,100.0,4,4,100.0,0.000,...,0.000,0.000000,≥ 20000 km,0.000000,,19,9,47.37,10,52.63
7,AE,1990,1,33.3,2,66.7,3,1,33.3,1.000,...,-12.500,0.000000,≥ 20000 km,0.000000,≥ 20000 km,29,15,51.72,14,48.28
8,AE,1991,4,26.7,11,73.3,15,7,46.7,3.750,...,71.429,0.000000,≥ 20000 km,0.000000,≥ 20000 km,32,7,21.88,25,78.13
9,AE,1992,6,17.6,28,82.4,34,22,64.7,4.333,...,23.217,0.000000,≥ 20000 km,0.000000,≥ 20000 km,65,18,27.69,47,72.31


### ***(4.2) Top k Institutions***

#### *(4.2.1) Top Destination Institutions - not run*

In [ ]:
#@title old
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations AS
SELECT
  origin_node,
  date_d,
  destination_node,
  top_rank,
  incoming_flows
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY origin_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2022
    GROUP BY
      origin_node, destination_node, date_d
    HAVING incoming_flows > 1
  ) AS flows
) AS ranked_flows
WHERE top_rank <= 5
GROUP BY
  origin_node,
  date_d,
  destination_node,
  top_rank,
  incoming_flows
ORDER BY
  origin_node, date_d;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations
where destination_node = 'grid.1001.0'
ORDER BY   
  origin_node,
  date_d,
  top_rank ASC
LIMIT 1000;

#### *(4.2.2) Top Source Institutions - not run*

In [ ]:
#@title old
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources AS
SELECT
  destination_node,
  date_d,
  origin_node,
  top_rank,
  incoming_flows
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY destination_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2022
    GROUP BY
      origin_node, destination_node, date_d
    HAVING incoming_flows > 1
  ) AS flows
) AS ranked_flows
WHERE top_rank <= 5
GROUP BY
  origin_node,
  date_d,
  destination_node,
  top_rank,
  incoming_flows
ORDER BY
  destination_node, date_d;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources
where origin_node = 'grid.1001.0'
ORDER BY   
  origin_node,
  date_d,
  top_rank ASC
LIMIT 1000;

#### *(4.2.2) Top Source/Destination Institutions*

In [31]:
#@title new
%%bigquery --project $project_id

# TOP 5 DESTINATIONS

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_destinations AS

--Top 5 overall destination institutions for each source institution:
WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, p2 as date_d, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022_country
  GROUP BY source_country, destination_country, p2
),

destination_ranks AS (
  SELECT source_country, destination_country, date_d, count_80_22,
    ROW_NUMBER() OVER (PARTITION BY date_d, source_country ORDER BY  count_80_22 DESC) AS destination_rank
  FROM source_destination_counts
),

top_destinations_per_source AS (
  SELECT source_country, destination_country, date_d, count_80_22 AS destination_count_1980_2022, destination_rank AS destination_rank_1980_2022,
  FROM destination_ranks
  WHERE destination_rank <= 5 AND count_80_22 > 1
)

SELECT 
  source_country,
  destination_country,
  date_d,
  destination_count_1980_2022,
  destination_rank_1980_2022
FROM top_destinations_per_source
ORDER BY date_d, destination_rank_1980_2022, destination_count_1980_2022, source_country ASC;


# TOP 5 SOURCES

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_sources AS

--Top 5 overall source institutions for each destination institution:

WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, p2 as date_d, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022_country
  GROUP BY source_country, destination_country, p2
),

source_ranks AS (
  SELECT source_country, destination_country, date_d, count_80_22,
    ROW_NUMBER() OVER (PARTITION BY  date_d, destination_country ORDER BY count_80_22 DESC) AS source_rank
  FROM source_destination_counts
),

top_sources_per_destination AS (
  SELECT source_country, destination_country, date_d, count_80_22 AS source_count_1980_2022, source_rank as source_rank_1980_2022
  FROM source_ranks
  WHERE source_rank <= 5 AND count_80_22 > 1
)

SELECT 
  source_country,
  destination_country,
  date_d,
  source_count_1980_2022,
  source_rank_1980_2022
FROM top_sources_per_destination
ORDER BY date_d, source_rank_1980_2022, source_count_1980_2022, destination_country ASC;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project $project_id
-- check tables
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_country_top_5_destinations
ORDER BY source_country, date_d, destination_rank_1980_2022
LIMIT 200;

In [ ]:
%%bigquery --project $project_id
-- check tables
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_country_top_5_sources
ORDER BY destination_country, date_d, source_rank_1980_2022
LIMIT 200;

#### *(4.2.3) Top Overall Destination - Source Institutions*

In [ ]:
%%bigquery --project $project_id

# TOP 5 DESTINATIONS

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_destinations_overall AS

--Top 5 overall destination countrys for each source country:

WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022 
  GROUP BY source_country, destination_country
),

destination_ranks AS (
  SELECT source_country, destination_country, count_80_22,
    RANK() OVER (PARTITION BY source_country ORDER BY count_80_22 DESC) AS destination_rank
  FROM source_destination_counts
),

top_destinations_per_source AS (
  SELECT source_country, destination_country, count_80_22 AS destination_count_1980_2022, destination_rank AS destination_rank_1980_2022,
  FROM destination_ranks
  WHERE destination_rank <= 5 AND count_80_22 > 1
)

SELECT 
  source_country,
  destination_country,
  destination_count_1980_2022,
  destination_rank_1980_2022
FROM top_destinations_per_source
ORDER BY destination_rank_1980_2022, destination_count_1980_2022, source_country ASC;


# TOP 5 SOURCES

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_country_top_5_sources_overall AS

--Top 5 overall source countrys for each destination country:

WITH source_destination_counts AS (
  SELECT unit1 as source_country, unit2 as destination_country, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022 
  GROUP BY source_country, destination_country
),

source_ranks AS (
  SELECT source_country, destination_country, count_80_22,
    RANK() OVER (PARTITION BY destination_country ORDER BY count_80_22 DESC) AS source_rank
  FROM source_destination_counts
),

top_sources_per_destination AS (
  SELECT source_country, destination_country, count_80_22 AS source_count_1980_2022, source_rank as source_rank_1980_2022
  FROM source_ranks
  WHERE source_rank <= 5 AND count_80_22 > 1
)

SELECT 
  source_country,
  destination_country,
  source_count_1980_2022,
  source_rank_1980_2022
FROM top_sources_per_destination
ORDER BY source_rank_1980_2022, source_count_1980_2022, destination_country ASC;

### ***(4.3) Productivity***

#### *(4.3.1) Author Productivity*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_cumulative_pubs_1980_2022_country AS

WITH ordered_publications AS (
  SELECT DISTINCT
    researcher_id as researcher_ids,
    year,
    COUNT(DISTINCT pub_id) as n_pubs, 
    ROW_NUMBER() OVER(PARTITION BY researcher_id ORDER BY year) AS publication_number
  FROM
    cshdimensionstest.test.au_pub_history_1980_2022_country
  GROUP BY researcher_id,  year
)
SELECT
  researcher_ids,
  year,
  SUM(n_pubs) OVER(PARTITION BY researcher_ids ORDER BY year
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                   ) AS cumulative_publications
FROM
  ordered_publications
ORDER BY
  researcher_ids,
  year;

SELECT * 
FROM  cshdimensionstest.test.au_cumulative_pubs_1980_2022_country
WHERE researcher_ids = 'ur.01000000010.53'
ORDER BY year;

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(DISTINCT researcher_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022 -- 6,435,674

**Ok all good**

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_cum_pubs_country AS

SELECT 
  f.researcher_ids, 
  f.unit1, 
  f.unit2, 
  f.p1,
  f.p2,
  (
    SELECT 
      MAX(a.cumulative_publications)
    FROM 
      cshdimensionstest.test.au_cumulative_pubs_1980_2022_country a
    WHERE 
       a.researcher_ids = f.researcher_ids
      AND a.year = f.p2
  ) AS source_cumulative_pubs
FROM 
  cshdimensionstest.test.flows_1980_2022_country f;

SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country; -- 3,031,288

In [44]:
%%bigquery --project $project_id
SELECT * FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country ORDER BY researcher_ids, p1, p2, unit1, unit2 LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,source_cumulative_pubs
0,ur.010000000201.99,TW,JP,2014,2020,4
1,ur.01000000021.08,JP,CA,2010,2013,3
2,ur.01000000021.08,CA,EG,2013,2016,4
3,ur.01000000021.08,EG,SA,2016,2021,9
4,ur.010000000667.71,PK,MX,2020,2022,2
5,ur.010000001435.52,IT,DE,2016,2021,9
6,ur.01000000145.32,US,CN,2001,2004,4
7,ur.01000000145.32,CN,GB,2004,2009,19
8,ur.01000000145.32,GB,SE,2009,2011,29
9,ur.01000000175.26,ES,DE,2010,2018,15


In [45]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_deciles_country AS
SELECT 
  researcher_ids,
  unit1, 
  unit2, 
  p1, 
  p2,
  source_cumulative_pubs,
  CASE
    WHEN source_cumulative_pubs BETWEEN 1 AND 3 THEN 'Productivity group id: 1 | Range (pubs): 1-3'
    WHEN source_cumulative_pubs BETWEEN 4 AND 10 THEN 'Productivity group id: 2 | Range (pubs): 4-10'
    WHEN source_cumulative_pubs BETWEEN 11 AND 25 THEN 'Productivity group id: 3 | Range (pubs): 11-25'
    WHEN source_cumulative_pubs BETWEEN 26 AND 50 THEN 'Productivity group id: 4 | Range (pubs): 26-50'
    WHEN source_cumulative_pubs BETWEEN 51 AND 70 THEN 'Productivity group id: 5 | Range (pubs): 51-70'
    WHEN source_cumulative_pubs BETWEEN 71 AND 100 THEN 'Productivity group id: 6 | Range (pubs): 71-100'
    WHEN source_cumulative_pubs BETWEEN 101 AND 250 THEN 'Productivity group id: 7 | Range (pubs): 101-250'
    WHEN source_cumulative_pubs BETWEEN 251 AND 500 THEN 'Productivity group id: 8 | Range (pubs): 251-500'
    WHEN source_cumulative_pubs BETWEEN 501 AND 1000 THEN 'Productivity group id: 9 | Range (pubs): 501-1000'
    ELSE 'Productivity group id: 10 | Range (pubs): 1001+' END AS pub_group
FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country
GROUP BY  
  researcher_ids,
  unit1, 
  unit2, 
  p1, 
  p2, 
  source_cumulative_pubs;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_pub_deciles_country
ORDER BY researcher_ids, p1, p2, unit1, unit2
LIMIT 20;


Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,source_cumulative_pubs,pub_group
0,ur.010000000201.99,TW,JP,2014,2020,4,Productivity group id: 2 | Range (pubs): 4-10
1,ur.01000000021.08,JP,CA,2010,2013,3,Productivity group id: 1 | Range (pubs): 1-3
2,ur.01000000021.08,CA,EG,2013,2016,4,Productivity group id: 2 | Range (pubs): 4-10
3,ur.01000000021.08,EG,SA,2016,2021,9,Productivity group id: 2 | Range (pubs): 4-10
4,ur.010000000667.71,PK,MX,2020,2022,2,Productivity group id: 1 | Range (pubs): 1-3
5,ur.010000001435.52,IT,DE,2016,2021,9,Productivity group id: 2 | Range (pubs): 4-10
6,ur.01000000145.32,US,CN,2001,2004,4,Productivity group id: 2 | Range (pubs): 4-10
7,ur.01000000145.32,CN,GB,2004,2009,19,Productivity group id: 3 | Range (pubs): 11-25
8,ur.01000000145.32,GB,SE,2009,2011,29,Productivity group id: 4 | Range (pubs): 26-50
9,ur.01000000175.26,ES,DE,2010,2018,15,Productivity group id: 3 | Range (pubs): 11-25


In [46]:
%%bigquery --project $project_id
SELECT DISTINCT pub_group FROM  cshdimensionstest.test.flows_1980_2022_pub_deciles_country order by pub_group;

Query is running:   0%|          |

Downloading:   0%|          |

,pub_group
0,Productivity group id: 1 | Range (pubs): 1-3
1,Productivity group id: 10 | Range (pubs): 1001+
2,Productivity group id: 2 | Range (pubs): 4-10
3,Productivity group id: 3 | Range (pubs): 11-25
4,Productivity group id: 4 | Range (pubs): 26-50
5,Productivity group id: 5 | Range (pubs): 51-70
6,Productivity group id: 6 | Range (pubs): 71-100
7,Productivity group id: 7 | Range (pubs): 101-250
8,Productivity group id: 8 | Range (pubs): 251-500
9,Productivity group id: 9 | Range (pubs): 501-1000


#### *(4.3.2) Author Productivity Groups*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_groups_country AS

WITH outgoing AS (
SELECT unit1 as node, p2 as date_d, 
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 1 | Range (pubs): 1-3') AS outgoing_pub_1_3_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 2 | Range (pubs): 4-10') AS outgoing_pub_4_10_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 3 | Range (pubs): 11-25') AS outgoing_pub_11_25_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 4 | Range (pubs): 26-50') AS outgoing_pub_26_50_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 5 | Range (pubs): 51-70') AS outgoing_pub_51_70_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 6 | Range (pubs): 71-100') AS outgoing_pub_71_100_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 7 | Range (pubs): 101-250') AS outgoing_pub_101_250_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 8 | Range (pubs): 251-500') AS outgoing_pub_251_500_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 9 | Range (pubs): 501-1000') AS outgoing_pub_501_1000_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 10 | Range (pubs): 1001+') AS outgoing_pub_1001_7797_y
     FROM  cshdimensionstest.test.flows_1980_2022_pub_deciles_country
GROUP BY unit1, p2 )
, incoming AS (
  SELECT unit2 as node, p2 as date_d, 
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 1 | Range (pubs): 1-3') AS incoming_pub_1_3_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 2 | Range (pubs): 4-10') AS incoming_pub_4_10_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 3 | Range (pubs): 11-25') AS incoming_pub_11_25_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 4 | Range (pubs): 26-50') AS incoming_pub_26_50_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 5 | Range (pubs): 51-70') AS incoming_pub_51_70_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 6 | Range (pubs): 71-100') AS incoming_pub_71_100_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 7 | Range (pubs): 101-250') AS incoming_pub_101_250_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 8 | Range (pubs): 251-500') AS incoming_pub_251_500_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 9 | Range (pubs): 501-1000') AS incoming_pub_501_1000_y,
       COUNTIF(IFNULL(pub_group, '') = 'Productivity group id: 10 | Range (pubs): 1001+') AS incoming_pub_1001_7797_y
     FROM cshdimensionstest.test.flows_1980_2022_pub_deciles_country
GROUP BY unit2, date_d
) ,
age_aggregated AS 
(
  SELECT a.*, b.incoming_pub_1_3_y, b.incoming_pub_4_10_y, b.incoming_pub_11_25_y,
  b.incoming_pub_26_50_y, b.incoming_pub_51_70_y, b.incoming_pub_71_100_y, b.incoming_pub_101_250_y,
  b.incoming_pub_251_500_y, b.incoming_pub_501_1000_y, b.incoming_pub_1001_7797_y
  FROM outgoing a
  LEFT JOIN incoming b 
    ON a.node=b.node 
    AND a.date_d=b.date_d
)
SELECT *
FROM age_aggregated
ORDER BY node, date_d;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_pub_groups_country
ORDER BY node, date_d
LIMIT 50;

#### *(4.3.3) Productivity Statistics*

In [50]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_stats_country AS

WITH 
outgoing AS (
  SELECT 
    unit1 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(source_cumulative_pubs), 0) AS sum_outgoing_pubs,
    IFNULL(STDDEV(source_cumulative_pubs), 0) AS stdev_outgoing_pubs,
    IFNULL(APPROX_QUANTILES(source_cumulative_pubs, 3)[OFFSET(1)], 0) AS median_outgoing_pubs
  FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country
  GROUP BY unit1, p2
),
incoming AS (
  SELECT 
    unit2 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(source_cumulative_pubs), 0) AS sum_incoming_pubs,
    IFNULL(STDDEV(source_cumulative_pubs), 0) AS stdev_incoming_pubs,
    IFNULL(APPROX_QUANTILES(source_cumulative_pubs, 3)[OFFSET(1)], 0) AS median_incoming_pubs
  FROM cshdimensionstest.test.flows_1980_2022_cum_pubs_country
  GROUP BY unit2, p2
),
age_aggregated AS (
  SELECT 
    a.node,
    a.move_year,
 --   a.num_researchers AS outflows,
 --   b.num_researchers AS inflows,
    ROUND(IFNULL(a.sum_outgoing_pubs / a.num_researchers, 0), 3) AS mean_outgoing_pubs,
    ROUND(IFNULL(a.sum_outgoing_pubs / (a.num_researchers * a.num_researchers), 0), 3) AS mean_normalized_outgoing_pubs,
    IFNULL(median_outgoing_pubs, 0) AS median_outgoing_pubs,
    IFNULL(sum_outgoing_pubs, 0) AS sum_outgoing_pubs,
    IFNULL(stdev_outgoing_pubs, 0) AS stdev_outgoing_pubs,
    ROUND(IFNULL(b.sum_incoming_pubs / b.num_researchers, 0), 3) AS mean_incoming_pubs,
    ROUND(IFNULL(b.sum_incoming_pubs / (b.num_researchers * b.num_researchers), 0), 3) AS mean_normalized_incoming_pubs,
    IFNULL(median_incoming_pubs, 0) AS median_incoming_pubs,
    IFNULL(sum_incoming_pubs, 0) AS sum_incoming_pubs,
    IFNULL(stdev_incoming_pubs, 0) AS stdev_incoming_pubs
FROM outgoing a
LEFT JOIN incoming b
  ON       a.node = b.node 
  AND a.move_year = b.move_year
)
SELECT * 
FROM age_aggregated
ORDER BY node, move_year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_pub_stats_country
ORDER BY node, move_year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,mean_outgoing_pubs,mean_normalized_outgoing_pubs,median_outgoing_pubs,sum_outgoing_pubs,stdev_outgoing_pubs,mean_incoming_pubs,mean_normalized_incoming_pubs,median_incoming_pubs,sum_incoming_pubs,stdev_incoming_pubs
0,AE,1988,11.000,11.000,11,11,0.000000,2.000,0.667,2,6,0.000000
1,AE,1990,3.000,3.000,3,3,0.000000,6.500,3.250,2,13,6.363961
2,AE,1991,4.250,1.063,2,17,3.304038,8.182,0.744,3,90,9.217572
3,AE,1992,3.500,0.583,2,21,2.810694,10.179,0.364,4,285,12.049446
4,AE,1993,2.250,0.563,2,9,0.500000,8.000,0.533,3,120,7.955232
5,AE,1994,9.000,1.125,2,72,9.304377,13.320,0.533,5,333,18.741042
6,AE,1995,11.900,1.190,5,119,9.882083,9.733,0.649,5,146,6.169819
7,AE,1996,7.308,0.562,2,95,6.663140,10.476,0.499,2,220,18.930449
8,AE,1997,3.755,0.077,1,184,5.198279,9.830,0.112,4,865,13.915228
9,AE,1998,11.895,0.626,4,226,11.584927,14.870,0.647,5,342,16.496017


In [52]:
%%bigquery --project $project_id
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_pub_stats_country
WHERE node = 'US'
ORDER BY move_year ASC;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,mean_outgoing_pubs,mean_normalized_outgoing_pubs,median_outgoing_pubs,sum_outgoing_pubs,stdev_outgoing_pubs,mean_incoming_pubs,mean_normalized_incoming_pubs,median_incoming_pubs,sum_incoming_pubs,stdev_incoming_pubs
0,US,1980,1.855,0.015,1,230,1.064532,1.708,0.012,1,234,1.165714
1,US,1981,2.414,0.008,2,775,1.528244,2.582,0.008,2,865,1.672560
2,US,1982,3.004,0.006,2,1577,2.389763,3.206,0.005,2,1962,2.201619
3,US,1983,3.406,0.004,2,2595,2.708056,3.808,0.005,2,3107,2.824531
4,US,1984,3.877,0.004,2,3943,3.293815,4.263,0.003,2,5239,3.251938
5,US,1985,4.477,0.004,2,5681,3.929423,4.537,0.003,2,7155,3.804241
6,US,1986,4.959,0.003,2,8128,4.616520,5.187,0.003,3,10022,4.619117
7,US,1987,5.453,0.003,2,11610,5.737857,5.494,0.002,3,16295,4.716454
8,US,1988,6.593,0.003,3,16930,8.838074,6.052,0.002,3,21368,5.642676
9,US,1989,7.262,0.002,3,21836,7.475910,6.442,0.002,3,25292,5.386983


This mean normalized calculation provides an indication of the overall productivity of the researchers that have joined the institution in a given calendar year. This indicator can help to assess the potential impact of their research on the group's output.

A value of 0.005 would mean that, on average, each researcher who has joined the group has had 0.005 publications before joining. Again, this value should be interpreted in the context of the specific group and field of research, but a low value could suggest that the group is not attracting highly productive or accomplished researchers.

Note that this indicator is highly affected by outliers.

In [ ]:
# merging the indicators
%%bigquery --project $project_id
CREATE OR REPLACE TABLE  cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_production   AS 
SELECT a.*,
    IFNULL(sum_outgoing_pubs, 0) AS total_outgoing_pubs,
    IFNULL(mean_outgoing_pubs, 0) AS mean_outgoing_pubs,
    IFNULL(mean_normalized_outgoing_pubs, 0) AS mean_normalized_outgoing_pubs,
    IFNULL(median_outgoing_pubs, 0) AS median_outgoing_pubs,
    IFNULL(stdev_outgoing_pubs, 0) AS stdev_outgoing_pubs,
    IFNULL(sum_incoming_pubs, 0) AS total_incoming_pubs,
    IFNULL(mean_incoming_pubs, 0) AS mean_incoming_pubs,
    IFNULL(mean_normalized_incoming_pubs, 0) AS mean_normalized_incoming_pubs,
    IFNULL(median_incoming_pubs, 0) AS median_incoming_pubs,
    IFNULL(stdev_incoming_pubs, 0) AS stdev_incoming_pubs
FROM cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_distances a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_pub_stats_country b ON a.node=b.node AND a.date_d=b.move_year;

**We consciously decide to not display any citation impact metrics.**

# END

#### *(4.4) Retention Rates*

To calculate the Scientists' retention rate per calendar year, I follow these steps:

    1. Identify the number of scientists affiliated in each institution in each year.
    2. Identify the number of scientists who were affiliated in the same institution in the previous year.
    3. Calculate the retention rate as the number of scientists who remained in the same institution divided by the total number of scientists enrolled in that institution.
    4. Repeat the process for each year and each institution.

This procedure calculates the retention rate of scientists affiliated with an institution from one year to the next. The retention rate is defined as *the ratio of the number of scientists affiliated with an institution in a given year to the number of scientists affiliated with the same institution in the previous year*.

The interpretation of the retention rate:

*A value of 1 means that the institution was able to retain the same number of scientists as the previous year, while a value greater than 1 indicates an increase in the number of affiliated scientists, and a value less than 1 indicates a decrease. The output can be used to assess the ability of institutions to retain their affiliated scientists over time*

This code calculates the retention rate of scientists for each institution and year, which is defined as the proportion of scientists who are affiliated with the institution in the current year and were also affiliated with the same institution in the previous year.

Advantages of this indicator include:

    It provides a clear picture of how well institutions are retaining their existing scientists.
    It is a simple indicator that can be easily understood.

Limitations of this indicator include:

    It does not take into account new scientists who may have joined the institution in the current year, so it may not provide a complete picture of the institution's ability to attract scientists.
    It assumes that the number of scientists affiliated with an institution in any given year is a good proxy for the institution's ability to attract and retain scientists. This may not always be the case, as factors such as funding, working conditions, and location can also play a significant role in attracting and retaining scientists.
    It also does not account for scientists who have left the institution but have not left the field entirely, which may lead to an over-estimation of the institution's retention rate.

The formula for the retention rate is calculated as:

retention_rate = affiliated_scientists_prev_year / affiliated_scientists

where:

    affiliated_scientists_prev_year represents the number of scientists affiliated with the institution in the previous year
    affiliated_scientists represents the number of scientists affiliated with the institution in the current year.

In [ ]:
#@title Retention test
%%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_retention AS
WITH scientist_retention 
AS (
SELECT
research_orgs,
year,
researcher_ids,
LAG(research_orgs) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_research_orgs,
LAG(year) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_year,
FROM FROM cshdimensionstest.test.simple_sequence_1980_2022
WHERE year >= 1980 AND year <= 2022
),
retention_rate AS (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS remaining_scientists
FROM scientist_retention
WHERE research_orgs = prev_research_orgs AND year = prev_year + 2 -- checks how many scientists remained affiliated in the previous 2 years
GROUP BY research_orgs, year
),
retention_rate_with_affiliation AS (
SELECT
retention_rate.research_orgs,
retention_rate.year,
retention_rate.remaining_scientists,
scientist_affiliation.affiliated_scientists,
CASE 
  WHEN scientist_affiliation.affiliated_scientists = 0 THEN 0
  ELSE retention_rate.remaining_scientists / scientist_affiliation.affiliated_scientists
END AS retention_rate
FROM retention_rate
JOIN (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS affiliated_scientists
FROM cshdimensionstest.test.simple_sequence_1980_2022
WHERE year >= 1980 AND year <= 2022
GROUP BY research_orgs, year
) AS scientist_affiliation
ON retention_rate.research_orgs = scientist_affiliation.research_orgs 
AND retention_rate.year = scientist_affiliation.year)
SELECT 
research_orgs,
year,
affiliated_scientists,
remaining_scientists,
retention_rate
FROM retention_rate_with_affiliation;

* One thing that we notice is that the non-migrant percentage is negative.

* The reason why this happens is that we calculate the population number for each year and do not take into account the population that stayed from the previous year. We need to make some adjustments.

* To count only the distinct cumulative population for each year, you could add a column to your query that identifies whether a scientist is still affiliated with the same institution in the next year. Then, you can sum up the number of scientists who are still affiliated with the same institution and divide that by the total number of affiliated scientists to get the cumulative non-migrant percentage. 

* non-mobile = what is the percentage of non-mobile researchers who have never left up until that point in time

-- we need to change the `non_migrant_percentage `indicator and align it with the `retention_rate` indicators


In [ ]:
#@title Vis snippet
client = bigquery.Client()

# Make the query
df = pd.io.gbq.read_gbq('''
select * 
from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
where node in (
  select node
  from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
  group by node
  having sum(total_flows) > 50000
  order by sum(total_flows) desc)
''', project_id=project_id, dialect='standard')

# Average outgoing 
# Create a plotly scatter plot
fig = go.Figure()

for node in df['node'].unique():
    node_df = df[df['node'] == node]
    fig.add_trace(go.Scatter(x=node_df['date_d']
                             , y=node_df['percentage_outflows'] 
                             , mode='lines+markers'
                             , name=node
                             , marker=dict(color=node_df['total_flows'], showscale=False, colorscale='Blues', opacity=0.5)
                             , hovertemplate='Year: %{x}<br>' + 'Incoming Academic Age: %{y}<br>' + 'Total Flows: %{marker.color:.2f}<br>' + 'Node: ' + node + '<br><extra></extra>'))

# Add a shared color axis
fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title="Total Flows",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            len=0.5,
            tickangle=-45,
            tickmode='array',
            tickvals=[0, 500, 1000, 5000, 10000, 50000]
        ),
        showscale=True,
    ),
    title="Average Academic Age by Year and Institution",
    xaxis_title="Year",
    yaxis_title="Average Normalized Incoming Academic Age",
    showlegend=True,
)

# Show the plot
pio.show(fig)

# save the plot in plotly graph studio and edit it there
#!pip install chart-studio
#import chart_studio.plotly as py
#py.sign_in(username='Ferreir4', api_key='40YuWFKy73EGkgEjddBA')
# save the plot as an HTML file in your Google Drive folder
#url = py.plot(fig, filename='my_plot', auto_open=False)

# **Data Exports**

#### Countries' Metadata

In [54]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.country_coordinates` 
"""
countries_metadata = client.query(sql).to_dataframe()
countries_metadata.head(10)

# save the dataset
countries_metadata.to_csv('countries_metadata.csv', index_label='row_no', encoding = 'utf-8-sig')
#files.download('organisations_metadata.csv')

#from google.colab import drive
#drive.mount('/content/drive')
!cp countries_metadata.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

#### Organisations' Edges

In [55]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_flows` 
ORDER BY geoid_o, geoid_d, date_d
"""
country_edges = client.query(sql).to_dataframe()
country_edges.head(10)

country_edges.to_csv('country_edges.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_edges.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

#### Organisations' Indicators

In [56]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_flow_statistics_with_production` 
ORDER BY node, date_d
"""
country_indicators = client.query(sql).to_dataframe()
#country_indicators.head(10)

country_indicators.to_csv('country_indicators.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [57]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_age_groups_country` 
ORDER BY node, date_d
"""
country_indicators_age_group_counts = client.query(sql).to_dataframe()
country_indicators_age_group_counts.head(10)

country_indicators_age_group_counts.to_csv('country_indicators_age_group_counts.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_age_group_counts.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [58]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_pub_groups_country` 
ORDER BY node, date_d
"""
country_indicators_pub_group_counts = client.query(sql).to_dataframe()
country_indicators_pub_group_counts.head(10)

country_indicators_pub_group_counts.to_csv('country_indicators_pub_group_counts.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_pub_group_counts.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [59]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_top_5_sources` 
ORDER BY destination_country, date_d, source_rank_1980_2022
"""
country_indicators_top_5_sources = client.query(sql).to_dataframe()
country_indicators_top_5_sources.head(10)

country_indicators_top_5_sources.to_csv('country_indicators_top_5_sources_pairs.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_top_5_sources_pairs.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [60]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_country_top_5_destinations` 
ORDER BY source_country, date_d, destination_rank_1980_2022
"""
country_indicators_top_5_destinations= client.query(sql).to_dataframe()
country_indicators_top_5_destinations.head(10)

country_indicators_top_5_destinations.to_csv('country_indicators_top_5_destinations_pairs.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp country_indicators_top_5_destinations_pairs.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

# PART III - Coverage

1. Validation of trajectories based on ORCID information
2. This is currently work in progress
